# Mongo DB task B

In this task we have two tasks. In the first task we create a two in the mongoDB database and in the second task we have to fetch data from the collection using the find query.

## Task B0: Creating a database named 'project_sohail' using PyMongo

Firstly we have to perform initialization and connection to the MongoDB server using python. This is done using pymongo which is a mongoDB api for python.

In [41]:
from pymongo import MongoClient

client = MongoClient('localhost')

db = client.project_sohail  #creating a database with the name project_sohail


In [1]:
import pandas as pd

In [2]:
df_climate_historic = pd.read_csv("climate_historic.csv")
df_hotspot_historic = pd.read_csv("hotspot_historic.csv")

In [16]:
df_climate_historic.head()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,ID
0,948700,31/12/2016,19,56.8,7.9,11.1,0.00I,1
1,948700,2/01/2017,15,50.7,9.2,13.0,0.02G,2
2,948700,3/01/2017,16,53.6,8.1,15.0,0.00G,3
3,948700,4/01/2017,24,61.6,7.7,14.0,0.00I,4
4,948700,5/01/2017,24,62.3,7.0,13.0,0.00I,5


In [4]:
df_hotspot_historic.head()

,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
0,-37.966,145.051,2017-12-27T04:16:51,78,27/12/2017,68
1,-35.541,143.311,2017-12-27T00:02:15,82,27/12/2017,63
2,-35.554,143.307,2017-12-27T00:02:15,67,27/12/2017,53
3,-35.543,143.316,2017-12-27T00:02:14,86,27/12/2017,67
4,-37.708,145.100,2017-12-25T04:29:08,80,25/12/2017,54


In [8]:
# this part of the code is to crete unique ID's for the climate_historic table
id_climate = []
for i in range(1,len(df_climate_historic)+1):
    id_climate.append(i)
df_climate_historic["ID"] = id_climate    

In [17]:
#this part of the code is to create reference ID's for the hotspot_historic table
id_hotspot_historic = []
for i in df_hotspot_historic.index:
    for j in df_climate_historic.index:
        if df_hotspot_historic['date'][i] == df_climate_historic['date'][j]:
            id_hotspot_historic.append(df_climate_historic['ID'][j])


In [18]:
df_hotspot_historic['Reference_ID'] = id_hotspot_historic

## Task B1: Creating two collections using the csv files in the project_sohail database

Now we have to load the data into the database. In MongoDB all the data is stored in the database in BSON format. Hence firstly we convert all the data we have into dictionary format and then load it into mongoDB collection.

In [42]:
#creating a collection named climateData and loading climate_historic.csv to it
db.climateData.delete_many({})  #Deleting any collection with this name
records = df_climate_historic.to_dict(orient = 'records') #converting the csv data into dictionary format
db.climateData.insert_many(records)

In [43]:
#loading hotspot_historic.csv file to the mongoDB database
records = df_hotspot_historic.to_dict(orient='records')
db.hotspot_historic.delete_many({}) #deleting any collection with this name
db.hotspot_historic.insert_many(records) #Here a collection named hotspot_historic would be created


As of now we have a database named __'project_sohail'__ and in this database we have two collections named __'climateData'__ and __'hotspot_historic'__

## Task B2: Fetching of data using PyMongo

Now we have to start fetching the data which is the second task in task B.

The standard way of fetching data from the collections is by using the find query if mongoDB. Using this we perform the fetching of data. Syntax of how to use this is found on the mongoDB official documentation.

__Solution 2a: Fetching climate data for 10th of December 2017__

In [44]:
#finding the record for the date match
for i in db.climateData.find({'date':'10/12/2017'}):
    print(i)
    print("\n\n")


{'_id': ObjectId('5cd8dfb43adc9af5ef087595'), 'station': 948702, 'date': '10/12/2017', 'air_temperature_celcius': 17, 'relative_humidity': 53.5, 'windspeed_knots': 7.3, 'max_wind_speed': 14.0, 'precipitation ': ' 0.00I'}





__ Solution 2b: Finding the latitude, longitude, surface temperature (°C), and confidence 
when the surface temperature (°C) was between 65 °C and 100 °C__

Using regular expressions in mongoDB we can fetch the data from the mongoDB database. Here we use __'\$gt'__ which fetches all the data with a higher value than specified and __'\$lt'__ which would fetch values lesser than specified value. Syntax of how to use regular expressions could be found in the mongoDB official documentation.

In [45]:
#now finding records satisfying the condition in the problem statement

for i in db.hotspot_historic.find({'surface_temperature_celcius': { "$gt": 65, "$lt":100  } },{'latitude':1, 'longitude':1,'surface_temperature_celcius':1,'confidence':1,'_id':0}):
    print(i)
    print("\n\n")

{'latitude': -37.966, 'longitude': 145.05100000000002, 'confidence': 78, 'surface_temperature_celcius': 68}



{'latitude': -35.543, 'longitude': 143.316, 'confidence': 86, 'surface_temperature_celcius': 67}



{'latitude': -37.875, 'longitude': 142.51, 'confidence': 93, 'surface_temperature_celcius': 73}



{'latitude': -37.613, 'longitude': 149.305, 'confidence': 95, 'surface_temperature_celcius': 75}



{'latitude': -37.624, 'longitude': 149.314, 'confidence': 90, 'surface_temperature_celcius': 66}



{'latitude': -38.056999999999995, 'longitude': 144.211, 'confidence': 93, 'surface_temperature_celcius': 73}



{'latitude': -37.95, 'longitude': 142.366, 'confidence': 92, 'surface_temperature_celcius': 70}



{'latitude': -36.282, 'longitude': 146.157, 'confidence': 100, 'surface_temperature_celcius': 71}



{'latitude': -37.634, 'longitude': 149.237, 'confidence': 100, 'surface_temperature_celcius': 71}



{'latitude': -37.605, 'longitude': 149.30200000000002, 'confidence': 98, 'sur

__Solution 2c: Finding date, surface temperature (°C), air temperature (°C), relative humidity and
max wind speed on 15th and 16th of December 2017.__

Now lies a slightly difficult task in this question. Here we have to fetch data from two different collections. The collections named climateData would have 'air temperature', 'relative humidity' and 'max wind speed' and the collection hotspot_historic has 'surface temperature'.

In [53]:
print("total number of unique dates", len(df_climate_historic['date'].unique()))
print("total number of unique records",len(df_climate_historic))

total number of unique dates 366
total number of unique records 366


For overcomming this problem a small trick has been used here. If observed 'date' is a common tag in both the data and in the climateData collection for one date there is only one record which can be seen in the above output. Hence a list is used for storing hotspot_historic and climateData records and then based on date they are sorted. After the sorting is done they are merged based on date and then displayed.

The first step thus is to fetch data for the dates 15 and 16 December 2017 from both the collections

In [54]:
# for finding the date and surface temperature based on the date
hotspot_list = []
climate_list = []
for i in db.hotspot_historic.find( {'$or': [{'date': '15/12/2017'},{'date': '16/12/2017'}]},{'date':1, '_id':0, 'surface_temperature_celcius':1}):
    hotspot_list.append(i)
    
# for finding the air tempurature relative humidity and max wind speed based on the date 
for i in db.climateData.find({'$or':[{'date':'15/12/2017'},{'date':'16/12/2017'}]},{'_id':0, 'air_temperature_celcius':1, 'relative_humidity':1, 'max_wind_speed':1, 'date':1}):
    climate_list.append(i)

Now that we have fetched the data and stored it in the list we would merge the list based on the date tag

In [49]:
hotspot_list_sort = sorted(hotspot_list, key = lambda i: i['date'])
climate_list_sort = sorted(climate_list, key = lambda i: i['date'])
hotspotindex = 0
for i in climate_list_sort:
    dateval = i['date']
    air_temp = i['air_temperature_celcius']
    for j in range(hotspotindex,len(hotspot_list_sort)):
        if(hotspot_list_sort[j]['date'] == dateval):
            hotspot_list_sort[j].update(i)
for i in hotspot_list_sort:
    print(i)

{'date': '15/12/2017', 'surface_temperature_celcius': 42, 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0}
{'date': '15/12/2017', 'surface_temperature_celcius': 36, 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0}
{'date': '15/12/2017', 'surface_temperature_celcius': 38, 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0}
{'date': '15/12/2017', 'surface_temperature_celcius': 40, 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0}
{'date': '16/12/2017', 'surface_temperature_celcius': 43, 'air_temperature_celcius': 18, 'relative_humidity': 53.7, 'max_wind_speed': 13.0}
{'date': '16/12/2017', 'surface_temperature_celcius': 33, 'air_temperature_celcius': 18, 'relative_humidity': 53.7, 'max_wind_speed': 13.0}
{'date': '16/12/2017', 'surface_temperature_celcius': 54, 'air_temperature_celcius': 18, 'relative_humidity': 53.7, 'max_wind_speed': 13.0}
{'date': '16/12/2017

__Solution 2d: Finding datetime, air temperature (°C), surface temperature (°C) and confidence
when the confidence is between 80 and 100.__

This question is very similar to the last question where we fetch data from different collections.

Hence in this solution too the same procedure is followed where in we fetch the data from the collections and store it in different list. Then we would sort the lists based on date and merge the lists.

__Some points to note in this solution are:__
* As we see that in the climate_historic.csv file we have the air temperature and date columns
* Also in the hotspot_historic.csv we have the confidence and date columns
* Hence for finding the solution we have to find the dates for the cvonfidence intervals and then find unique dates of them
* In the climate_historic.csv we can see that there is records which is unique for the dates 
* Hence for the unique dates we would fetch the records and then display the air temperature 
* Regular expressions are used for fetching the data, syntax and use or regular expressions can be found in mongoDB official docs.

In [55]:
lisun=[] #creating a list to store all the date values for confidence intervals between 80 and 100

hotspot_historic = []
climate_data = []

#appending the date values from hotspot_historic.csv file based on the conofidence interval range
for i in db.hotspot_historic.find({'confidence': {'$gt': 80, '$lt':100}},{'_id':0, 'datetime':1, 'surface_temperature_celcius':1, 'date':1, 'confidence':1}):
    if i['date'] not in lisun:
        lisun.append(i['date'])
    hotspot_historic.append(i) #print of datetime, confidence, date and surface temperature based on the confidence interval range
    
print("\n\n")


#print of date and air temperature for the specific confidence interval range
for i in db.climateData.find({'date': {'$in': lisun}}, {'_id':0, 'air_temperature_celcius':1, 'date':1}):
    climate_data.append(i)

Now that we have fetched the data and stored it in different lists we would sort the lists based on date and then merge them based on date itself.

In [21]:
hotspot_historic_sorted = sorted(hotspot_historic, key = lambda i: i['date'])
climate_data_sorted = sorted(climate_data, key = lambda i: i['date'])
hotspotindex = 0
for i in climate_data_sorted:
    dateval = i['date']
    air_temp = i['air_temperature_celcius']
    for j in range(hotspotindex,len(hotspot_historic_sorted)):
        if(hotspot_historic_sorted[j]['date'] == dateval):
            hotspot_historic_sorted[j].update(i)
for i in hotspot_historic_sorted:
    print(i)

{'datetime': '2017-05-01T04:17:50', 'confidence': 94, 'date': '1/05/2017', 'surface_temperature_celcius': 103, 'air_temperature_celcius': 14}
{'datetime': '2017-05-01T04:14:30', 'confidence': 82, 'date': '1/05/2017', 'surface_temperature_celcius': 55, 'air_temperature_celcius': 14}
{'datetime': '2017-05-01T04:14:30', 'confidence': 82, 'date': '1/05/2017', 'surface_temperature_celcius': 55, 'air_temperature_celcius': 14}
{'datetime': '2017-05-01T04:14:20', 'confidence': 83, 'date': '1/05/2017', 'surface_temperature_celcius': 77, 'air_temperature_celcius': 14}
{'datetime': '2017-05-01T04:14:20', 'confidence': 82, 'date': '1/05/2017', 'surface_temperature_celcius': 62, 'air_temperature_celcius': 14}
{'datetime': '2017-05-01T04:14:20', 'confidence': 95, 'date': '1/05/2017', 'surface_temperature_celcius': 76, 'air_temperature_celcius': 14}
{'datetime': '2017-05-01T04:14:20', 'confidence': 85, 'date': '1/05/2017', 'surface_temperature_celcius': 59, 'air_temperature_celcius': 14}
{'datetime':

__Solution 2e: Find the top 10 records with the highest surface temperature (°C)__

In this question we have to find the records from hotspot_historic for the highest surface temperatures.

For this we would use a list and store all the surface temperatures from the hotspot_historic colectio. then we would use basic principle of sorting and find the top 10 highest value records. 

Now using regular expressions we can fetch records from the collection for all the temperatures in the list

'\$in' regular expression is used in this solution. Syntax and use of this can be found in mongoDB official Documantation 

In [47]:

surtemplis = [] #this is a list to get the top 10 highest temperatures
for i in db.hotspot_historic.find({},{'_id':0,'surface_temperature_celcius':1}):
    surtemplis.append(i['surface_temperature_celcius'])
surtemplis = list(set(surtemplis))
surtemplis.sort(reverse=True)
surtemplis = surtemplis[:10] #here we have the top 10 highest temperatures

#now we display the data for the top 10 highest temperatures
for i in db.hotspot_historic.find({'surface_temperature_celcius': {'$in': surtemplis}},{'_id':0}):
    print(i)

{'latitude': -34.9938, 'longitude': 141.876, 'datetime': '2017-05-13T04:40:20', 'confidence': 100, 'date': '13/05/2017', 'surface_temperature_celcius': 120}
{'latitude': -36.2829, 'longitude': 145.825, 'datetime': '2017-05-08T04:20:10', 'confidence': 100, 'date': '8/05/2017', 'surface_temperature_celcius': 115}
{'latitude': -36.4057, 'longitude': 140.9806, 'datetime': '2017-05-04T04:44:50', 'confidence': 100, 'date': '4/05/2017', 'surface_temperature_celcius': 119}
{'latitude': -36.6088, 'longitude': 145.2514, 'datetime': '2017-05-03T04:09:40', 'confidence': 100, 'date': '3/05/2017', 'surface_temperature_celcius': 115}
{'latitude': -36.9318, 'longitude': 143.0907, 'datetime': '2017-05-01T04:14:20', 'confidence': 100, 'date': '1/05/2017', 'surface_temperature_celcius': 122}
{'latitude': -36.9348, 'longitude': 143.09799999999998, 'datetime': '2017-05-01T04:14:20', 'confidence': 87, 'date': '1/05/2017', 'surface_temperature_celcius': 117}
{'latitude': -38.1665, 'longitude': 143.062, 'date

__Solution 2f: Find the number of fire in each day. You are required to only display the
total number of fire and the date in the output__

To find the number of fires in a day we have to check the number of records for each date in hotspot_historic collection and the number of records for each day gives us the total number of fires.

Our first step is the to find the number of records for each date in hotspot_historic collection and then find the count of the dates

In [98]:

hotspotdate = hotspot_historic_sorted[0]['date']
firecount = 0
firelist = []
for i in hotspot_historic_sorted:
    if(i['date'] == hotspotdate):
        firecount+=1
    else:
        tempdic = {}
        tempdic['date'] = hotspotdate
        tempdic['total_number_fire_count'] = firecount
        firecount = 1
        hotspotdate = i['date']
        firelist.append(tempdic)
tempdic = {}
tempdic['date'] = hotspotdate
tempdic['total_number_fire_count'] = firecount
firelist.append(tempdic)

However out task is not complete here. There could be some dates where no fire incidents would have occured hence we have to find all the dates for which the record is in consideration. For this we use the climate_historic dataframe and use the unique funciton.

In [99]:
uniqueDates = (sorted(df_climate_historic['date'].unique()))

Now that we know all the dates for which the data is being considered we now would perform a lookup weather on the date a fire has occured and if not then we append a record which says 0 fire incidents occured.

This can be done by appending all the unique dates of the firelist to a list. After this we use all the dates in consideration and perform a lookup to see wether a record for the date exists in firelist. If not then we append a record for the date with 0 fire incident count.

In [101]:
firelist_dates = []
for i in firelist:
    firelist_dates.append(i['date'])
# finding the fire occurences for 0 fire occurences
for i in uniqueDates:
    if i not in firelist_dates:
        tempdic = {}
        tempdic['date'] = i
        tempdic['total_number_fire_count'] = 0
        firelist.append(tempdic)
        
firelist_sorted = sorted(firelist, key = lambda i: i['date'])  

for i in firelist_sorted:
    print(i)


{'date': '1/01/2018', 'total_number_fire_count': 0}
{'date': '1/02/2017', 'total_number_fire_count': 0}
{'date': '1/03/2017', 'total_number_fire_count': 0}
{'date': '1/04/2017', 'total_number_fire_count': 0}
{'date': '1/05/2017', 'total_number_fire_count': 8}
{'date': '1/06/2017', 'total_number_fire_count': 1}
{'date': '1/07/2017', 'total_number_fire_count': 0}
{'date': '1/08/2017', 'total_number_fire_count': 1}
{'date': '1/09/2017', 'total_number_fire_count': 0}
{'date': '1/10/2017', 'total_number_fire_count': 2}
{'date': '1/11/2017', 'total_number_fire_count': 0}
{'date': '1/12/2017', 'total_number_fire_count': 0}
{'date': '10/01/2017', 'total_number_fire_count': 0}
{'date': '10/02/2017', 'total_number_fire_count': 0}
{'date': '10/03/2017', 'total_number_fire_count': 3}
{'date': '10/04/2017', 'total_number_fire_count': 0}
{'date': '10/05/2017', 'total_number_fire_count': 35}
{'date': '10/06/2017', 'total_number_fire_count': 0}
{'date': '10/07/2017', 'total_number_fire_count': 0}
{'da

__Solution 2g: Finding the average surface temperature (°C) for each day. You are required to
only display average surface temperature (°C) and the date in the output.__

Here we have to find the average temperatures for each date in the hotspot_historic collection. This can be done by grouping the temperatures based on date and then finding the average of the temperatures for a specific date. The group by is done using aggrgate function and then average is found for each date

In [106]:

for i in db.hotspot_historic.aggregate([{'$group': {'_id': "$date", 'avgtemp': {'$avg' : '$surface_temperature_celcius'}}}]):
    print(i)

{'_id': '7/03/2017', 'avgtemp': 64.0}
{'_id': '8/03/2017', 'avgtemp': 51.5}
{'_id': '9/03/2017', 'avgtemp': 46.666666666666664}
{'_id': '14/03/2017', 'avgtemp': 65.6}
{'_id': '17/03/2017', 'avgtemp': 59.5}
{'_id': '18/03/2017', 'avgtemp': 79.33333333333333}
{'_id': '24/03/2017', 'avgtemp': 49.0}
{'_id': '26/03/2017', 'avgtemp': 56.88235294117647}
{'_id': '19/03/2017', 'avgtemp': 65.57142857142857}
{'_id': '28/03/2017', 'avgtemp': 60.925925925925924}
{'_id': '3/04/2017', 'avgtemp': 58.44444444444444}
{'_id': '4/04/2017', 'avgtemp': 62.57303370786517}
{'_id': '1/04/2017', 'avgtemp': 46.714285714285715}
{'_id': '7/04/2017', 'avgtemp': 50.69230769230769}
{'_id': '12/04/2017', 'avgtemp': 52.69565217391305}
{'_id': '13/04/2017', 'avgtemp': 58.57983193277311}
{'_id': '14/04/2017', 'avgtemp': 61.94444444444444}
{'_id': '6/03/2017', 'avgtemp': 60.5}
{'_id': '15/04/2017', 'avgtemp': 59.57971014492754}
{'_id': '19/04/2017', 'avgtemp': 54.16}
{'_id': '20/04/2017', 'avgtemp': 56.58064516129032}
{'_

However in the assignment it is specified that the use of aggregate is not allowed hence mannually we can group the data based on dates by storing it in different lists and then the mean function from numpy library can be used to find the average.

In [107]:
import numpy
hotspot_historic = []
for i in db.hotspot_historic.find():
    hotspot_historic.append(i)
unique_date = hotspot_historic[0]['date']
avg_temp_date = []
avg_temp = []
# appending the dates in a unique pattern and then finding the average using numpy mean
for i in hotspot_historic:
    if(i['date'] == unique_date):
        avg_temp.append(i['surface_temperature_celcius'])
        
    else:
        tempdic = {}
        tempdic['date'] = unique_date
        tempdic['average_surface_temperature'] = numpy.mean(avg_temp)
        avg_temp = []
        avg_temp_date.append(tempdic)
        unique_date = i['date']
        avg_temp.append(i['surface_temperature_celcius'])
# appending the mean for the last record in the list
tempdic = {}
tempdic['date'] = unique_date
tempdic['average_surface_temperature'] = numpy.mean(avg_temp)
avg_temp_date.append(tempdic)
for i in avg_temp_date:
    print(i)

{'date': '27/12/2017', 'average_surface_temperature': 62.75}
{'date': '25/12/2017', 'average_surface_temperature': 54.0}
{'date': '24/12/2017', 'average_surface_temperature': 32.0}
{'date': '21/12/2017', 'average_surface_temperature': 46.0}
{'date': '16/12/2017', 'average_surface_temperature': 57.8}
{'date': '15/12/2017', 'average_surface_temperature': 39.0}
{'date': '14/12/2017', 'average_surface_temperature': 70.0}
{'date': '13/12/2017', 'average_surface_temperature': 60.0}
{'date': '12/12/2017', 'average_surface_temperature': 44.0}
{'date': '10/12/2017', 'average_surface_temperature': 46.0}
{'date': '9/12/2017', 'average_surface_temperature': 58.25}
{'date': '8/12/2017', 'average_surface_temperature': 50.6}
{'date': '30/11/2017', 'average_surface_temperature': 52.41935483870968}
{'date': '29/11/2017', 'average_surface_temperature': 60.625}
{'date': '28/11/2017', 'average_surface_temperature': 42.0}
{'date': '23/11/2017', 'average_surface_temperature': 58.8}
{'date': '22/11/2017', 'a